In [ ]:
# get data in order first, then plot the ones dependent on time bc i need them to be in that specific row

# important libs and constants
import plot_utils as p
import numpy as np
import csv
G = 6.674e-11  # N*m^2/Kg^2
Msolar = 1.989e30  # kg
Rsolar = 6.957e8  # in m
c = 299792458.0  # m/s

# containers for all solutions (for combined plots)
all_q = []
all_a_spin = []
all_kQ = []

with open("possible_output.csv", "r") as o:
    reader = csv.DictReader(o)
    for row in reader:
        sol_id = row["ID"]

        data = np.load(f"solutions/{sol_id}.npz")
        t = data["t"]
        f = data["f"]
        Omega = data["Omega"]

        # read metadata from csv
        m1 = float(row["m_WR/M_Sun"])
        m2 = float(row["m_companion/M_Sun"])
        a0 = float(row["initial separation a0 (m)"])
        Omega0 = float(row["initial spin Omega0 (Hz)"])
        f0 = float(row["initial frequency f0 (Hz)"])
        k = float(row["k"])
        Q = float(row["Q"])
        R1 = float(row["WR star Radius R_WR"])
        lifetime = float(row["lifetime (years)"])
        T_TF = float(row["tidal function timescale (years)"])
        f_final = float(row["f_final (Hz)"])
        Omega_final = float(row["Omega_final (Hz)"])
        J = float(row["final angular momentum J (kg m^2/s)"])
        a_spin = float(row["spin parameter a_spin"])
        a_final = float(row["final separation a_final (m)"])
        a_min_si = float(row["minimum separation a_min (m)"])
        q = m1/m2

        # plot f/f0(t)
        p.plot2d(t, f/f0, xlabel='t [years]',
                 ylabel='f/f0', title=f"f/f0 vs t for m1={format(m1/Msolar, '.0e')} M_sun, m2={format(m2/Msolar, '.0e')} M_sun, k={format(k, '.0e')}, Q={format(Q, '.0e')}, a0={format(a0, '.2e')} m, Ω0={format(Omega0, '.2e')} Hz, lifetime={format(lifetime, '.0e')} years, T_TF={format(T_TF, '.0e')} years, f0={format(f0, '.2e')} Hz, f_final={format(f_final, '.2e')} Hz, Omega_final={format(Omega_final, '.2e')} Hz", legend='f/f0(t)', colour=None, axis_font='Tahoma', axis_size=12, title_font='Georgia', title_size=16, legend_size=10)
        # plot Omega(t)
        p.plot2d(t, Omega, xlabel='t [years]',
                 ylabel='Ω [Hz]', title=f"Ω vs t for m1={format(m1/Msolar, '.0e')} M_sun, m2={format(m2/Msolar, '.0e')} M_sun, k={format(k, '.0e')}, Q={format(Q, '.0e')}, a0={format(a0, '.2e')} m, Ω0={format(Omega0, '.2e')} Hz, lifetime={format(lifetime, '.0e')} years, T_TF={format(T_TF, '.0e')} years, f0={format(f0, '.2e')} Hz, f_final={format(f_final, '.2e')} Hz, Omega_final={format(Omega_final, '.2e')} Hz", legend='Ω(t)', colour=None, axis_font='Tahoma', axis_size=12, title_font='Georgia', title_size=16, legend_size=10)
        all_q.append(q)
        all_a_spin.append(a_spin)
        all_kQ.append(k/Q)

In [ ]:
# combined plots for all solutions
all_q = np.array(all_q)
all_a_spin = np.array(all_a_spin)
all_kQ = np.array(all_kQ)

# plot a_spin vs q
p.plot2d(q, a_spin, xlabel='q = m1/m2',
         ylabel='a_spin', title=f"a_spin vs q for m1={format(m1/Msolar, '.0e')} M_sun, m2={format(m2/Msolar, '.0e')} M_sun, k={format(k, '.0e')}, Q={format(Q, '.0e')}, a0={format(a0, '.2e')} m, Ω0={format(Omega0, '.2e')} Hz, lifetime={format(lifetime, '.0e')} years, T_TF={format(T_TF, '.0e')} years, f0={format(f0, '.2e')} Hz, f_final={format(f_final, '.2e')} Hz, Omega_final={format(Omega_final, '.2e')} Hz", legend='a_spin(q)', colour=None, axis_font='Tahoma', axis_size=12, title_font='Georgia', title_size=16, legend_size=10)
# plot a_spin vs k/Q !!!! MIGHT ADD SEPARATION LIMIT
p.plot2d(k/Q, a_spin, xlabel='k/Q',
         ylabel='a_spin', title=f"a_spin vs k/Q for m1={format(m1/Msolar, '.0e')} M_sun, m2={format(m2/Msolar, '.0e')} M_sun, k={format(k, '.0e')}, Q={format(Q, '.0e')}, a0={format(a0, '.2e')} m, Ω0={format(Omega0, '.2e')} Hz, lifetime={format(lifetime, '.0e')} years, T_TF={format(T_TF, '.2e')} years, f0={format(f0, '.2e')} Hz, f_final={format(f_final, '.2e')} Hz, Omega_final={format(Omega_final, '.2e')} Hz", legend='a_spin(k/Q)', colour=None, axis_font='Tahoma', axis_size=12, title_font='Georgia', title_size=16, legend_size=10)